In [5]:
import cv2
import imutils
import pandas as pd
import numpy as np
import pytesseract
from datetime import datetime
import csv
from geopy.geocoders import Nominatim

''' for creating csv file'''
headers = ["Vehicle No." , "DATE" , "Time","State","Location"]
List = []

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

img = cv2.imread('c1.jpg',cv2.IMREAD_COLOR)
img = cv2.resize(img, (600,400) )

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
gray = cv2.bilateralFilter(gray, 13, 15, 15) 

edged = cv2.Canny(gray, 30, 200) 
contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None

for c in contours:
    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)
 
    if len(approx) == 4:
        screenCnt = approx
        break

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
     detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

mask = np.zeros(gray.shape,np.uint8)
new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
new_image = cv2.bitwise_and(img,img,mask=mask)

(x, y) = np.where(mask == 255)
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
Cropped = gray[topx:bottomx+1, topy:bottomy+1]


img = cv2.resize(img,(500,300))
Cropped = cv2.resize(Cropped,(400,200))
img1 = cv2.cvtColor(Cropped, cv2.COLOR_BGR2RGB)

'''Date TIME'''
a = (pytesseract.image_to_string(img1))
d= datetime.date(datetime.now())
t= datetime.time(datetime.now())

cn=a
cm=[]
ab = list(a)

kl=[]
i=0
for i in range(0,2):
    #print(ab[i])
    kl.append(ab[i])

km="".join(kl)
#print(km)
with open("listofstateab.csv", "r") as scoreFile:
    scoreFileReader = csv.reader(scoreFile)
    a = km
    for row in scoreFileReader:
        if row[2] == a:
            #print(row)
            #print(row[1])
            abc = row[1]



# calling the nominatim tool
geoLoc = Nominatim(user_agent="GetLoc")

# passing the coordinates
locname = geoLoc.reverse("10.9421,76.9290")
dc = locname.address 


cm=[cn,d,t,abc,dc]
List.append(cm)
'''CREATING CSV FILE WITH LOCATION'''
with open("c1.csv", "w") as Tollno:
    toll = csv.writer(Tollno)
    toll.writerow(headers)
    toll.writerows(List)








cv2.imshow("display", img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [21]:
import csv
from datetime import datetime
Vn=""
d=""
t=""
vn2 = ""
d2 = ""
t2 = ""

with open("c1.csv",mode ='r' ) as c1:

    csv_reader = csv.reader(c1,delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 2:
            Vn=row[0]
            d=row[1]
            t=row[2]
            break
        line_count += 1

with open("c2.csv",mode='r') as c2:
    csv_reader =csv.reader(c2,delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 2:
            vn2 = row[0]
            d2 = row[1]
            t2 = row[2]
            break
        line_count += 1


x= d +  ' ' +t
y= d2+ ' ' + t2

z=datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")
q=datetime.strptime(y, "%Y-%m-%d %H:%M:%S.%f")

dif= q-z



timetaken=int(dif.total_seconds())
#print("Time Taken in Seconds ="+" "+str(timetaken)+" seconds")
distance= 1000     # in meters
speed = distance/timetaken
kmph=speed*3.6

print("The car crossed the first camera at "+ str(z))
print("The car crossed the second camera at "+ str(q))
print("Time Taken in Seconds ="+" "+str(timetaken)+" seconds")
if kmph>60:
    print("The driver has over speeded in the blind spot.")
else:
    print("The driver has not over speeded in the blind spot.")
print("speed = {:.2f} kmph".format(kmph)) 





The car entered the first camera at 2022-11-10 22:29:57.187340
The car entered the second camera at 2022-11-10 22:31:06.118140
Time Taken in Seconds = 68 seconds
The driver has not over speeded in the blind spot.
speed = 52.94 kmph
